# Imports and dataset load

In [ ]:
!pip install kaggle
!pip install kaggle==1.5.6

In [4]:
from google.colab import files
uploaded = files.upload()

!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle competitions download -c newsclass01

In [ ]:
!unzip newsclass01.zip

In [ ]:
!pip install transformers

In [10]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup

# Tokenizer, model

In [12]:
df_train = pd.read_csv('./train.csv')
index_train, index_val = train_test_split(df_train.index.values, 
                                          test_size=0.1, 
                                          random_state=0, 
                                          stratify=df_train['source'].values)

df_train['type'] = ['-'] * df_train.shape[0]
df_train.loc[index_train, 'type'] = 'train'
df_train.loc[index_val, 'type'] = 'val'

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('youscan/ukr-roberta-base')
                                          
encoded_train = tokenizer.batch_encode_plus(
    df_train[df_train['type']=='train']['text'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512,
    return_tensors='pt'
)

encoded_val = tokenizer.batch_encode_plus(
    df_train[df_train['type']=='val']['text'].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512,
    return_tensors='pt'
)

dataset_train = TensorDataset(encoded_train['input_ids'],
                              encoded_train['attention_mask'],
                              torch.tensor(df_train[df_train['type']=='train']['source'].values))

dataset_val = TensorDataset(encoded_val['input_ids'],
                            encoded_val['attention_mask'],
                            torch.tensor(df_train[df_train['type']=='val']['source'].values))

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=8)

dataloader_val = DataLoader(dataset_val, 
                            sampler=SequentialSampler(dataset_val), 
                            batch_size=8)

In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
import random

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
model = RobertaForSequenceClassification.from_pretrained("youscan/ukr-roberta-base",
                                                        num_labels=7,
                                                        output_attentions=False,
                                                        output_hidden_states=False)
model = model.to(device)

In [16]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)                  
epochs = 5
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# Training

In [20]:
!mkdir roberta_models

In [21]:
model_dir = 'roberta_models'

In [ ]:
def evaluate(dataloader_val):
    model.eval()
    loss_val_total = 0
    predictions = [] 
    true_vals = []
    for batch in dataloader_val:        
        batch = tuple(b.to(device) for b in batch)
        with torch.no_grad():        
            outputs = model(input_ids=batch[0],
                            attention_mask=batch[1],
                            labels=batch[2])            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = batch[2].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)    
    loss_val_avg = loss_val_total / len(dataloader_val) 
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)            
    return loss_val_avg, predictions, true_vals

In [22]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)        
        outputs = model(input_ids=batch[0],
                        attention_mask=batch[1],
                        labels=batch[2])        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})                 
    torch.save(model.state_dict(), f'{model_dir}/roberta_{epoch}.model')        
    tqdm.write(f'\nEpoch {epoch}')    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')    
    val_loss, predictions, true_vals = evaluate(dataloader_val)    
    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = true_vals.flatten()
    val_f1 = f1_score(labels_flat, preds_flat, average='macro')
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Macro): {val_f1}')


Epoch 1
Training loss: 0.34918391778233593
Validation loss: 0.16324815748051827
F1 Score (Macro): 0.9544720693584817



Epoch 2
Training loss: 0.14822618047393996
Validation loss: 0.28152425970036016
F1 Score (Macro): 0.9392513143274378



Epoch 3
Training loss: 0.08376286601901434
Validation loss: 0.24079045938908408
F1 Score (Macro): 0.9530948187757274



Epoch 4
Training loss: 0.04541719102075309
Validation loss: 0.20508784489976983
F1 Score (Macro): 0.9574805130481652



Epoch 5
Training loss: 0.017808258622014593
Validation loss: 0.18940651475608086
F1 Score (Macro): 0.9592551299842381



# Predict

In [24]:
df_test = pd.read_csv('./test_without_target.csv')

In [ ]:
encoded_test = tokenizer.batch_encode_plus(
    df_test.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512,
    return_tensors='pt'
)

dataset_test = TensorDataset(encoded_test['input_ids'],
                             encoded_test['attention_mask'],
                             torch.tensor(df_test.Id.values))

dataloader_test = DataLoader(dataset_test, 
                              sampler=RandomSampler(dataset_test), 
                              batch_size=8)

In [ ]:
predict_model = RobertaForSequenceClassification.from_pretrained("youscan/ukr-roberta-base",
                                                      num_labels=7,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

predict_model.to(device)
predict_model.load_state_dict(torch.load(f'{model_dir}/roberta_epoch_5.model', map_location=torch.device('cpu')))

In [27]:
def predict(dataloader):
    predict_model.eval()
    predictions = []
    ids = []
    for batch in dataloader:
        ids.append(batch[2].numpy())
        batch = tuple(b.to(device) for b in batch)       
        with torch.no_grad():        
            outputs = predict_model(input_ids=batch[0],
                                  attention_mask=batch[1])            
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)        
    predictions = np.concatenate(predictions, axis=0)
    ids = np.concatenate(ids, axis=0)
    preds_flat = np.argmax(predictions, axis=1).flatten()
    return preds_flat, ids

In [ ]:
preds, ids = predict(dataloader_test)

result = zip(ids.tolist(), preds.tolist())
result = list(result)
result.sort(key=lambda tup: tup[0])
result = [[i[0], i[1]] for i in result]

In [31]:
res_file = pd.DataFrame(result, columns = ['Id', 'Predicted'])
res_file.to_csv('out.csv', index=False)